In [1]:
import time
import random
import pandas as pd
import numpy as np
import os

from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, MaxPooling2D, Conv2D, Dropout, Lambda, Dense, Flatten, Activation, Input, Embedding, BatchNormalization
from tensorflow.keras.initializers import glorot_normal, Zeros, TruncatedNormal
from tensorflow.keras.regularizers import l2


from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy


from tensorflow.keras.optimizers import Adam
from collections import defaultdict
import math

c:\Users\Admin\anaconda3\envs\ds6_rcmm\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [2]:
class FeaturesEmbedding(Layer):  
    '''
    임베딩 레이어입니다. 
    - 만약 피처(feature) 3개가 각각 10개, 20개, 30개의 고유값을 가진다면 feature_dims는 [10, 20, 30] 형태를 띄게 됩니다.
    - 전체 임베딩을 해야 할 개수는 10+20+30 = 60이므로 '60 x 임베딩_차원_크기'의 행렬이 생성되게 됩니다.
    '''
    def __init__(self, field_dims, embed_dim, **kwargs):
        super(FeaturesEmbedding, self).__init__(**kwargs)
        self.total_dim = sum(field_dims)
        self.embed_dim = embed_dim
        ## int 64로 바꿔주자
        self.offsets = np.array((0, *np.cumsum(field_dims)[:-1]), dtype=np.int64)
        self.embedding = tf.keras.layers.Embedding(input_dim=self.total_dim, output_dim=self.embed_dim)

    def build(self, input_shape):
        # 임베딩을 빌드하고 초기화합니다.
        self.embedding.build(input_shape)
        self.embedding.set_weights([tf.keras.initializers.GlorotUniform()(shape=self.embedding.weights[0].shape)])

    def call(self, x):
        # 들어온 입력의 임베딩을 가져니다.
        x = x + tf.constant(self.offsets)
        return self.embedding(x)

In [3]:
class MultiLayerPerceptron(Layer):  
    '''
    DNN 레이어입니다.
    - Tensorflow Keras에서는 Dense 레이어를 쌓아올린 구조입니다.
    - 필요에 따라 배치 정규화도 사용할 수 있습니다.
    '''
    def __init__(self, input_dim, hidden_units, activation='relu', l2_reg=0, dropout_rate=0, use_bn=False, init_std=0.0001, output_layer=True):
        super(MultiLayerPerceptron, self).__init__()
        self.dropout_rate = dropout_rate
        self.use_bn = use_bn
        hidden_units = [input_dim] + list(hidden_units)
        if output_layer:
            hidden_units += [1]
        # Dense layer를 쌓아올립니다.
        self.linears = [Dense(units, activation=None, kernel_initializer=tf.random_normal_initializer(stddev=init_std),
                              kernel_regularizer=tf.keras.regularizers.l2(l2_reg)) for units in hidden_units[1:]]
        # 활성화 함수를 세팅합니다.
        self.activation = tf.keras.layers.Activation(activation)
        # 필요하다면 배치정규화도 진행합니다.
        if self.use_bn:
            self.bn = [BatchNormalization() for _ in hidden_units[1:]]
        self.dropout = Dropout(dropout_rate)

    def call(self, inputs, training=False):
        x = inputs
        for i in range(len(self.linears)):
            # input data가 들어오면 layer를 돌면서 벡터 값을 가져오게 됩니다.
            x = self.linears[i](x)
            if self.use_bn:
                x = self.bn[i](x, training=training)
            # 각 layer마다 나온 벡터 값에 활성화 함수와 dropout을 적용시켜 비선형성 구조와 과적합을 방지합니다.
            x = self.activation(x)
            x = self.dropout(x, training=training)
        return x

In [4]:
class MultiHeadSelfAttention(Layer):  
    '''
    멀티 헤드 셀프 어텐션 레이어입니다.
    - 위에 작성한 수식과 같이 동작됩니다.
    - 필요에 따라 잔차 연결(residual connection)도 진행합니다.
    '''
    def __init__(self, att_embedding_size=8, head_num=2, use_res=True, scaling=False, seed=1024, **kwargs):
        if head_num <= 0:
            raise ValueError('head_num must be a int > 0')
        self.att_embedding_size = att_embedding_size
        self.head_num = head_num
        self.use_res = use_res
        self.seed = seed
        self.scaling = scaling
        super(MultiHeadSelfAttention, self).__init__(**kwargs)

    def build(self, input_shape):
        if len(input_shape) != 3:
            raise ValueError(
                "Unexpected inputs dimensions %d, expect to be 3 dimensions" % (len(input_shape)))
        embedding_size = int(input_shape[-1])
        # 쿼리에 해당하는 매트릭스입니다. 
        self.W_Query = self.add_weight(name='query', shape=[embedding_size, self.att_embedding_size * self.head_num],
                                       dtype=tf.float32,
                                       initializer=TruncatedNormal(seed=self.seed))
        # 키에 해당되는 매트릭스입니다.
        self.W_key = self.add_weight(name='key', shape=[embedding_size, self.att_embedding_size * self.head_num],
                                     dtype=tf.float32,
                                     initializer=TruncatedNormal(seed=self.seed + 1))
        # 값(value)에 해당되는 매트릭스입니다.
        self.W_Value = self.add_weight(name='value', shape=[embedding_size, self.att_embedding_size * self.head_num],
                                       dtype=tf.float32,
                                       initializer=TruncatedNormal(seed=self.seed + 2))
        # 필요하다면 잔차 연결도 할 수 있습니다.
        if self.use_res:
            self.W_Res = self.add_weight(name='res', shape=[embedding_size, self.att_embedding_size * self.head_num],
                                         dtype=tf.float32,
                                         initializer=TruncatedNormal(seed=self.seed))

        super(MultiHeadSelfAttention, self).build(input_shape)

    def call(self, inputs, **kwargs):
        if K.ndim(inputs) != 3:
            raise ValueError("Unexpected inputs dimensions %d, expect to be 3 dimensions" % (K.ndim(inputs)))
        
        # 입력이 들어오면 쿼리, 키, 값(value)에 매칭되어 각각의 값을 가지고 옵니다.
        querys = tf.tensordot(inputs, self.W_Query, axes=(-1, 0))  
        keys = tf.tensordot(inputs, self.W_key, axes=(-1, 0))
        values = tf.tensordot(inputs, self.W_Value, axes=(-1, 0))

        # 헤드 개수에 따라 데이터를 분리해줍니다.
        querys = tf.stack(tf.split(querys, self.head_num, axis=2))
        keys = tf.stack(tf.split(keys, self.head_num, axis=2))
        values = tf.stack(tf.split(values, self.head_num, axis=2))
        
        # 쿼리와 키를 먼저 곱해줍니다. 위 이미지의 식 (5)와 같습니다.
        inner_product = tf.matmul(querys, keys, transpose_b=True)
        if self.scaling:
            inner_product /= self.att_embedding_size ** 0.5
        self.normalized_att_scores =  tf.nn.softmax(inner_product)
        
        # 쿼리와 키에서 나온 어텐션 값을 값(value)에 곱해줍니다. 식 (6)과 같습니다.
        result = tf.matmul(self.normalized_att_scores, values)
        # 식 (7)과 같이 쪼개어진 멀테 헤드를 모아줍니다.
        result = tf.concat(tf.split(result, self.head_num, ), axis=-1)
        result = tf.squeeze(result, axis=0) 

        if self.use_res:
            result += tf.tensordot(inputs, self.W_Res, axes=(-1, 0))
        result = tf.nn.relu(result)
        
        # 그 결과 값을 리턴합니다.

        return result

    def compute_output_shape(self, input_shape):

        return (None, input_shape[1], self.att_embedding_size * self.head_num)

    def get_config(self, ):
        config = {'att_embedding_size': self.att_embedding_size, 'head_num': self.head_num, 'use_res': self.use_res,'seed': self.seed}
        base_config = super(MultiHeadSelfAttention, self).get_config()
        base_config.update(config)
        return base_config

# 바꿀 부분

이 밑에있는 모델 정의를 새로운 모델로 해주셔야합니다

In [5]:
class AutoIntMLP(Layer): 
    def __init__(self, field_dims, embedding_size, att_layer_num=3, att_head_num=4, att_res=True, 
                 dnn_hidden_units=(256, 128, 64), dnn_activation='relu',
                 l2_reg_dnn=1e-5, l2_reg_embedding=1e-5, dnn_use_bn=True, dnn_dropout=0.2, init_std=0.001):
        super(AutoIntMLP, self).__init__()
        self.embedding = FeaturesEmbedding(field_dims, embedding_size)
        self.num_fields = len(field_dims)
        self.embedding_size = embedding_size

        self.dnn = tf.keras.Sequential()
        for units in dnn_hidden_units:
            self.dnn.add(Dense(units, kernel_initializer=tf.random_normal_initializer(stddev=init_std)))
            if dnn_use_bn:
                self.dnn.add(BatchNormalization())
            self.dnn.add(Activation(dnn_activation))
            if dnn_dropout > 0:
                self.dnn.add(Dropout(dnn_dropout))
        
        self.int_layers = [MultiHeadSelfAttention(att_embedding_size=embedding_size, head_num=att_head_num, use_res=att_res) 
                           for _ in range(att_layer_num)]

        # [중요] Concat 후 결과를 하나로 합치는 최종 출력층 (에러 해결의 핵심)
        self.combine_dense = Dense(1, activation='sigmoid', kernel_initializer=tf.random_normal_initializer(stddev=init_std))

    def call(self, inputs, training=False):
        embed_x = self.embedding(inputs)
        
        # 1. Attention 경로
        att_input = embed_x
        for layer in self.int_layers:
            att_input = layer(att_input)
        att_output = Flatten()(att_input)

        # 2. DNN 경로
        dnn_embed = tf.reshape(embed_x, shape=(-1, self.embedding_size * self.num_fields))
        dnn_output = self.dnn(dnn_embed, training=training)

        # 3. [개선] 더하기 대신 연결(Concat) 방식 적용
        combined = tf.concat([att_output, dnn_output], axis=-1)
        
        return self.combine_dense(combined)

In [6]:
def get_DCG(ranklist, y_true):
    dcg = 0.0
    for i in range(len(ranklist)):
        item = ranklist[i]
        if item in y_true:
            dcg += 1.0 / math.log(i + 2)
    return  dcg

def get_IDCG(ranklist, y_true):
    idcg = 0.0
    i = 0
    for item in y_true:
        if item in ranklist:
            idcg += 1.0 / math.log(i + 2)
            i += 1
    return idcg

def get_NDCG(ranklist, y_true):
    '''NDCG 평가 지표'''
    ranklist = np.array(ranklist).astype(int)
    y_true = np.array(y_true).astype(int)
    dcg = get_DCG(ranklist, y_true)
    idcg = get_IDCG(y_true, y_true)
    if idcg == 0:
        return 0
    return round( (dcg / idcg), 5)

def get_hit_rate(ranklist, y_true):
    '''hitrate 평가 지표'''
    c = 0
    for y in y_true:
        if y in ranklist:
            c += 1
    return round( c / len(y_true), 5 )

In [7]:
def test_model(model, test_df):
    '''모델 테스트'''
    user_pred_info = defaultdict(list)
    total_rows = len(test_df)
    
    # 배치 사이즈가 정의되어 있지 않을 경우를 대비해 기본값 설정 (필요시 수정)
    # batch_size = 256 
    
    for i in range(0, total_rows, batch_size):
        # 정답(label) 열을 제외한 피처 추출
        features = test_df.iloc[i:i + batch_size, :-1].values
        y_pred = model.predict(features, verbose=False)
        
        for feature, p in zip(features, y_pred):
            # feature[:2]는 [user_id, movie_id] 형태의 배열입니다.
            u_i = feature[:2]
            
            # .item()을 사용하여 numpy array 내부의 값을 순수 파이썬 숫자로 추출합니다.
            u_id = int(u_i[0].item())
            m_id = int(u_i[1].item())
            
            # 예측 확률 값 p도 단일 숫자로 변환합니다.
            pred_score = float(p.item() if hasattr(p, 'item') else p)
            
            user_pred_info[u_id].append((m_id, pred_score))
            
    return user_pred_info

# 학습용 데이터 만들기

In [8]:
project_path = os.path.abspath(os.getcwd())
data_dir_nm = 'data'
movielens_dir_nm = 'ml-1m'
model_dir_nm = 'model'
data_path = f"{project_path}/{data_dir_nm}"

In [9]:
# 1. 데이터 불러오기
# csv 데이터이므로 read_csv로 가져옵니다.
movielens_rcmm = pd.read_csv("C:\\Users\\Admin\\autoint\\data\\movielens_rcmm_v2.csv", dtype=str)
print(movielens_rcmm.shape)
movielens_rcmm.head()

(1000209, 15)


,user_id,movie_id,movie_decade,movie_year,rating_year,rating_month,rating_decade,genre1,genre2,genre3,gender,age,occupation,zip,label
0,1,1193,1970s,1975,2000,12,2000s,Drama,no,no,F,1,10,48067,1
1,1,661,1990s,1996,2000,12,2000s,Animation,Children's,Musical,F,1,10,48067,0
2,1,914,1960s,1964,2000,12,2000s,Musical,Romance,no,F,1,10,48067,0
3,1,3408,2000s,2000,2000,12,2000s,Drama,no,no,F,1,10,48067,1
4,1,2355,1990s,1998,2001,1,2000s,Animation,Children's,Comedy,F,1,10,48067,1


In [10]:
label_encoders = {col: LabelEncoder() for col in movielens_rcmm.columns[:-1]} # label은 제외

for col, le in label_encoders.items():
    movielens_rcmm[col] = le.fit_transform(movielens_rcmm[col])

In [11]:
movielens_rcmm.head()

,user_id,movie_id,movie_decade,movie_year,rating_year,rating_month,rating_decade,genre1,genre2,genre3,gender,age,occupation,zip,label
0,0,189,6,55,0,3,0,7,17,15,0,0,2,1588,1
1,0,3374,8,76,0,3,0,2,2,8,0,0,2,1588,0
2,0,3615,5,44,0,3,0,11,12,15,0,0,2,1588,0
3,0,2503,9,80,0,3,0,7,17,15,0,0,2,1588,1
4,0,1374,8,78,1,0,0,2,2,2,0,0,2,1588,1


In [12]:
movielens_rcmm['label'] = movielens_rcmm['label'].astype(np.float32)

In [13]:
# 3. 학습 데이터와 테스트데이터로 분리, 0.2 정도로 분리
train_df, test_df = train_test_split(movielens_rcmm, test_size=0.2, random_state=42)

In [14]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 800167 entries, 416292 to 121958
Data columns (total 15 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   user_id        800167 non-null  int64  
 1   movie_id       800167 non-null  int64  
 2   movie_decade   800167 non-null  int64  
 3   movie_year     800167 non-null  int64  
 4   rating_year    800167 non-null  int64  
 5   rating_month   800167 non-null  int64  
 6   rating_decade  800167 non-null  int64  
 7   genre1         800167 non-null  int64  
 8   genre2         800167 non-null  int64  
 9   genre3         800167 non-null  int64  
 10  gender         800167 non-null  int64  
 11  age            800167 non-null  int64  
 12  occupation     800167 non-null  int64  
 13  zip            800167 non-null  int64  
 14  label          800167 non-null  float32
dtypes: float32(1), int64(14)
memory usage: 94.6 MB


In [15]:
# 필요 컬럼들과 레이블 정의
# 필드의 각 고유 개수를 정의하는 field_dims를 정의합니다. 이는  임베딩 때 활용됩니다. 
u_i_feature = ['user_id', 'movie_id']
meta_features = ['movie_decade', 'movie_year', 'rating_year', 'rating_month', 'rating_decade', 'genre1','genre2', 'genre3', 'gender', 'age', 'occupation', 'zip']
label = 'label'
field_dims = np.max(movielens_rcmm[u_i_feature + meta_features].astype(np.int64).values, axis=0) + 1
field_dims

array([6040, 3706,   10,   81,    4,   12,    1,   18,   18,   16,    2,
          7,   21, 3439])

In [16]:
# # 에포크, 학습률, 드롭아웃, 배치사이즈, 임베딩 크기 등 정의
# epochs=5
# learning_rate= 0.0001
# dropout= 0.4
# batch_size = 2048
# embed_dim= 16

# [변경 추천] 고성능을 위한 설정값
epochs = 10              # [변경] 5 -> 10 (조금 더 충분히 학습)
learning_rate = 0.001    # [변경] 0.0001 -> 0.001 (학습 속도 최적화)
dropout = 0.2            # [변경] 0.4 -> 0.2 (정보 손실을 줄임)
batch_size = 1024        # [변경] 2048 -> 1024 (더 세밀하게 가중치 업데이트)
embed_dim = 32           # [변경] 16 -> 32 (데이터의 특징을 더 풍부하게 담음)

# 고칠부분

이부분도 AutoIntMLP를 가져와주고 DNN 레이어가 붙은 파라미터를 추가해줍니다

In [17]:

class AutoIntMLPModel(Model):
    def __init__(self, field_dims, embedding_size, att_layer_num=3, att_head_num=2,
                 att_res=True, dnn_hidden_units=(32, 32), dnn_activation='relu',
                 l2_reg_dnn=0, l2_reg_embedding=1e-5, dnn_use_bn=False,
                 dnn_dropout=0.4, init_std=0.0001):
        super(AutoIntMLPModel, self).__init__()
        self.autoInt_layer = AutoIntMLP(
            field_dims=field_dims,
            embedding_size=embedding_size,
            att_layer_num=att_layer_num,
            att_head_num=att_head_num,
            att_res=att_res,
            dnn_hidden_units=dnn_hidden_units,
            dnn_activation=dnn_activation,
            l2_reg_dnn=l2_reg_dnn,
            l2_reg_embedding=l2_reg_embedding,
            dnn_use_bn=dnn_use_bn,
            dnn_dropout=dnn_dropout,
            init_std=init_std
        )

    def call(self, inputs, training=False):
        return self.autoInt_layer(inputs, training=training)

In [18]:
# # 모델 정의
# autoIntMLP_model = AutoIntMLPModel(
#     field_dims=field_dims,
#     embedding_size=embed_dim,
#     att_layer_num=3,
#     att_head_num=2,
#     att_res=True,
#     dnn_hidden_units=(32, 32),               # 추가: DNN 은닉층 구조
#     dnn_activation='relu',                  # 추가: 활성화 함수
#     l2_reg_dnn=0,
#     l2_reg_embedding=1e-5,
#     dnn_use_bn=False,
#     dnn_dropout=dropout,
#     init_std=0.0001
# )

# [수정 버전] 고성능 파라미터 적용
autoIntMLP_model = AutoIntMLPModel(
    field_dims=field_dims,
    embedding_size=32,               # [변경] 16(또는 기존값) -> 32
    att_layer_num=3,
    att_head_num=4,                  # [변경] 2 -> 4 (헤드 수 증가)
    att_res=True,
    dnn_hidden_units=(256, 128, 64), # [변경] (32, 32) -> (256, 128, 64)
    dnn_activation='relu',
    l2_reg_dnn=1e-5,                 # [변경] 0 -> 1e-5 (과적합 방지)
    l2_reg_embedding=1e-5,
    dnn_use_bn=True,                 # [변경] False -> True (학습 안정화)
    dnn_dropout=0.2,                 # [변경] 기존값 -> 0.2
    init_std=0.001                   # [변경] 0.0001 -> 0.001
)

In [19]:
# 옵티마이저, 오차함수 정의
optimizer = Adam(learning_rate=learning_rate)
loss_fn = BinaryCrossentropy(from_logits=False)

In [20]:
autoIntMLP_model.compile(optimizer=optimizer, loss=loss_fn, metrics=['binary_crossentropy'])

In [21]:
history = autoIntMLP_model.fit(train_df[u_i_feature + meta_features], train_df[label], epochs=epochs, batch_size=batch_size, validation_split=0.1)

Epoch 1/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 143s 190ms/step - binary_crossentropy: 0.5621 - loss: 0.5621 - val_binary_crossentropy: 0.5393 - val_loss: 0.5393
Epoch 2/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 121s 171ms/step - binary_crossentropy: 0.5285 - loss: 0.5285 - val_binary_crossentropy: 0.5298 - val_loss: 0.5298
Epoch 3/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 104s 147ms/step - binary_crossentropy: 0.5147 - loss: 0.5147 - val_binary_crossentropy: 0.5240 - val_loss: 0.5240
Epoch 4/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 115s 163ms/step - binary_crossentropy: 0.5020 - loss: 0.5020 - val_binary_crossentropy: 0.5195 - val_loss: 0.5195
Epoch 5/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 132s 188ms/step - binary_crossentropy: 0.4918 - loss: 0.4918 - val_binary_crossentropy: 0.5195 - val_loss: 0.5195
Epoch 6/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 114s 162ms/step - binary_crossentropy: 0.4826 - loss: 0.4826 - val_binary_crossentropy: 0.5171 - val_loss: 0.5171
Epoch 7/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 102s 145ms/step - binary_crossentrop

In [22]:
# 사용자에게 예측된 정보를 저장하는 딕셔너리 
user_pred_info = {}
# top10개
top = 10
# 테스트 값을 가지고 옵니다. 
mymodel_user_pred_info = test_model(autoIntMLP_model, test_df)
# 사용자마다 돌면서 예측 데이터 중 가장 높은 top 10만 가져옵니다. 
for user, data_info in tqdm(mymodel_user_pred_info.items(), total=len(mymodel_user_pred_info), position=0, leave=True):
    ranklist = sorted(data_info, key=lambda s : s[1], reverse=True)[:top]
    ranklist = list(dict.fromkeys([r[0] for r in ranklist]))
    user_pred_info[str(user)] = ranklist
# 원본 테스트 데이터에서 label이 1인 사용자 별 영화 정보를 가져옵니다.
test_data = test_df[test_df['label']==1].groupby('user_id')['movie_id'].apply(list)

100%|██████████| 6035/6035 [00:00<00:00, 35839.57it/s]


In [23]:
mymodel_ndcg_result = {}
mymodel_hitrate_result = {}

# 모델 예측값과 원본 테스트 데이터를 비교해서 어느정도 성능이 나왔는지 NDCG와 Hitrate를 비교합니다.

# NDCG
for user, data_info in tqdm(test_data.items(), total=len(test_data), position=0, leave=True):
    mymodel_pred = user_pred_info.get(str(user))

    testset = list(set(np.array(data_info).astype(int)))
    mymodel_pred = mymodel_pred[:top]

    # NDCG 값 구하기
    user_ndcg = get_NDCG(mymodel_pred, testset)

    mymodel_ndcg_result[user] = user_ndcg

# Hitrate
for user, data_info in tqdm(test_data.items(), total=len(test_data), position=0, leave=True):
    mymodel_pred = user_pred_info.get(str(user))

    testset = list(set(np.array(data_info).astype(int)))
    mymodel_pred = mymodel_pred[:top]

    # hitrate 값 구하기
    user_hitrate = get_hit_rate(mymodel_pred, testset)

    # 사용자 hitrate 결과 저장
    mymodel_hitrate_result[user] = user_hitrate

  5%|▌         | 305/5994 [00:00<00:01, 3036.26it/s]

100%|██████████| 5994/5994 [00:00<00:00, 51508.03it/s]


In [24]:
print(" mymodel ndcg : ", round(np.mean(list(mymodel_ndcg_result.values())), 5))
print(" mymodel hitrate : ", round(np.mean(list(mymodel_hitrate_result.values())), 5))

#  mymodel ndcg :  0.66298
#  mymodel hitrate :  0.6331

 mymodel ndcg :  0.6777
 mymodel hitrate :  0.64179


In [25]:
# import os

# # 1. 'model' 폴더가 현재 경로에 있는지 확인하고 없으면 만듭니다.
# model_dir = './model'
# if not os.path.exists(model_dir):
#     os.makedirs(model_dir)
#     print(f"✅ '{model_dir}' 폴더를 새로 생성했습니다.")

# # 2. 이제 다시 저장 코드를 실행합니다.
# autoIntMLP_model.save_weights('./model/autoIntMLP_model_weights.weights.h5')
# print("✅ 가중치 저장 완료!")

# # 1. 'model' 폴더가 현재 경로에 있는지 확인하고 없으면 만듭니다.
# model_dir = './model'
# if not os.path.exists(model_dir):
#     os.makedirs(model_dir)
#     print(f"✅ '{model_dir}' 폴더를 새로 생성했습니다.")

# # 2. 이제 다시 저장 코드를 실행합니다.
# autoIntMLP_model.save_weights('./model/autoIntMLP_model_weights.weights.h5')
# print("✅ 가중치 저장 완료!")

import os
save_path = r'C:\Users\Admin\autoint\model\autoIntMLP1_model_weights.weights.h5'
os.makedirs(os.path.dirname(save_path), exist_ok=True)
autoIntMLP_model.save_weights(save_path)
print(f"✅ 새 가중치 저장 성공: {save_path}")

✅ 새 가중치 저장 성공: C:\Users\Admin\autoint\model\autoIntMLP1_model_weights.weights.h5


In [26]:
import joblib 

# 모델 객체를 pickled binary file 형태로 저장
joblib.dump(label_encoders, './model/label_encoders1.pkl')

['./model/label_encoders1.pkl']